In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from _My_Functions import null_cols, rmse

importing Jupyter notebook from _My_Functions.ipynb


In [2]:
acc = pd.read_csv('../../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [3]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

# Data Cleaning and Imputation

## Getting rows that contain the desired target

In [4]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off

acc_train = copy.deepcopy(acc[(acc['loan_status'].str.contains('Paid|Default|Charged', na=False))])
acc_train = acc_train.reset_index()
acc_train = acc_train.drop('index', axis=1)
acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
# create target label

target_train = acc_train[['loan_status']]

target_train

,loan_status
0,Fully Paid
1,Fully Paid
2,Fully Paid
3,Fully Paid
4,Fully Paid
...,...
1348094,Fully Paid
1348095,Fully Paid
1348096,Charged Off
1348097,Fully Paid


In [6]:
target_train[['loan_status']].value_counts()

loan_status                                        
Fully Paid                                             1076751
Charged Off                                             268559
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
dtype: int64

In [7]:
# converting loan statuses to binary (0, 1)

target_train['loan_status_binary'] = 1

# create mask for clarity on what is being changed
mask_fully = target_train['loan_status'].str.contains('Fully', na=False)

target_train.loc[mask_fully, 'loan_status_binary'] = 0

target_train = target_train.drop('loan_status', axis=1)

target_train

<ipython-input-7-1d183777ee03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_train['loan_status_binary'] = 1
C:\Users\tdcho\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
1348094,0
1348095,0
1348096,1
1348097,0


In [8]:
target_train[['loan_status_binary']].value_counts()

loan_status_binary
0                     1078739
1                      269360
dtype: int64

In [9]:
acc_train = acc_train.drop(['loan_status'], axis=1)

acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,N

## Splitting the dataset into train and prediction

In [10]:
# getting indices of half of the samples

target_nodwnsmpl_index_0 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==0].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_1 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==1].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_0and1 = target_nodwnsmpl_index_0 + target_nodwnsmpl_index_1
target_nodwnsmpl_index_0and1

[0,
 5,
 6,
 7,
 8,
 14,
 15,
 19,
 20,
 22,
 25,
 28,
 32,
 35,
 37,
 38,
 42,
 43,
 53,
 56,
 57,
 58,
 61,
 64,
 65,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 82,
 85,
 87,
 90,
 94,
 96,
 97,
 98,
 99,
 102,
 104,
 105,
 109,
 110,
 113,
 115,
 116,
 119,
 123,
 126,
 129,
 133,
 135,
 142,
 144,
 153,
 155,
 156,
 162,
 165,
 167,
 171,
 172,
 173,
 175,
 176,
 179,
 180,
 183,
 184,
 185,
 186,
 190,
 191,
 193,
 195,
 199,
 200,
 203,
 204,
 205,
 206,
 207,
 209,
 215,
 216,
 218,
 220,
 222,
 225,
 226,
 228,
 229,
 230,
 231,
 234,
 235,
 238,
 243,
 249,
 251,
 255,
 258,
 261,
 262,
 265,
 266,
 267,
 271,
 273,
 278,
 279,
 283,
 284,
 285,
 295,
 298,
 299,
 301,
 302,
 304,
 309,
 311,
 312,
 314,
 322,
 323,
 326,
 328,
 329,
 330,
 332,
 343,
 344,
 347,
 348,
 349,
 353,
 356,
 358,
 359,
 360,
 361,
 363,
 366,
 367,
 368,
 371,
 374,
 380,
 381,
 385,
 386,
 388,
 390,
 391,
 395,
 396,
 397,
 400,
 402,
 403,
 407,
 411,
 413,
 416,
 420,
 422,
 423,
 426,
 427

In [11]:
len(target_nodwnsmpl_index_0and1)

674050

In [12]:
# getting the other half of the indices

target_nodwnsmpl_pred_index = sorted(list(set(list(range(0,
                                               target_train['loan_status_binary'].count()))) -\
                                set(target_nodwnsmpl_index_0and1)))

target_nodwnsmpl_pred_index

[1,
 2,
 3,
 4,
 9,
 11,
 12,
 13,
 16,
 17,
 18,
 21,
 23,
 24,
 26,
 27,
 29,
 30,
 31,
 33,
 34,
 39,
 40,
 41,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 55,
 59,
 60,
 62,
 63,
 66,
 76,
 79,
 80,
 81,
 84,
 86,
 89,
 91,
 92,
 93,
 95,
 101,
 103,
 106,
 108,
 112,
 114,
 117,
 118,
 120,
 122,
 124,
 125,
 128,
 131,
 132,
 134,
 137,
 138,
 139,
 140,
 141,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 157,
 158,
 159,
 161,
 163,
 164,
 166,
 168,
 169,
 170,
 174,
 177,
 181,
 182,
 187,
 189,
 192,
 194,
 196,
 197,
 198,
 201,
 208,
 210,
 211,
 212,
 214,
 217,
 219,
 221,
 223,
 224,
 227,
 232,
 233,
 236,
 237,
 239,
 241,
 242,
 244,
 245,
 246,
 247,
 248,
 250,
 252,
 253,
 254,
 256,
 257,
 260,
 263,
 264,
 269,
 270,
 272,
 274,
 275,
 276,
 277,
 282,
 286,
 287,
 288,
 289,
 291,
 292,
 293,
 294,
 296,
 297,
 303,
 306,
 307,
 308,
 310,
 313,
 315,
 317,
 318,
 319,
 321,
 324,
 325,
 327,
 331,
 333,
 334,
 335,
 336,
 339,
 340,
 341,
 342,
 345,
 350,

In [13]:
len(target_nodwnsmpl_pred_index)

674049

In [14]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_1_train_basicimputation = acc_train.iloc[target_nodwnsmpl_index_0and1]
acc_1_train_basicimputation = acc_1_train_basicimputation.reset_index()
acc_1_train_basicimputation = acc_1_train_basicimputation.drop('index', axis=1)

target_1_train_basicimputation = target_train.iloc[target_nodwnsmpl_index_0and1]
target_1_train_basicimputation = target_1_train_basicimputation.reset_index()
target_1_train_basicimputation = target_1_train_basicimputation.drop('index', axis=1)


acc_1_pred_basicimputation = acc_train.iloc[target_nodwnsmpl_pred_index]
acc_1_pred_basicimputation = acc_1_pred_basicimputation.reset_index()
acc_1_pred_basicimputation = acc_1_pred_basicimputation.drop('index', axis=1)

target_1_pred_basicimputation = target_train.iloc[target_nodwnsmpl_pred_index]
target_1_pred_basicimputation = target_1_pred_basicimputation.reset_index()
target_1_pred_basicimputation = target_1_pred_basicimputation.drop('index', axis=1)


acc_1_train_basicimputation

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68476668,NaN,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,0.0,0.0,0.0,2.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,

In [15]:
target_1_train_basicimputation

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
674045,1
674046,1
674047,1
674048,1


In [16]:
acc_1_pred_basicimputation.to_csv("acc_1_pred_basicimputation.csv.gz",
                                   index=False,
                                   compression="gzip")

target_1_pred_basicimputation.to_csv("target_1_pred_basicimputation.csv.gz",
                                      index=False,
                                      compression="gzip")

In [17]:
# drop unneeded columns

acc_clean = acc_1_train_basicimputation.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                               'next_pymnt_d', 'out_prncp', 'out_prncp_inv',
                                               'pymnt_plan', 'recoveries', 'title',
                                               'total_pymnt_inv','total_rec_int',
                                               'total_rec_late_fee', 'total_rec_prncp', 'url',
                                               'hardship_flag', 'hardship_type', 'hardship_reason',
                                               'hardship_status', 'deferral_term', 'hardship_amount',
                                               'hardship_start_date', 'hardship_end_date',
                                               'payment_plan_start_date', 'hardship_length',
                                               'hardship_dpd', 'hardship_loan_status', 
                                               'orig_projected_additional_accrued_interest',
                                               'hardship_payoff_balance_amount', 
                                               'hardship_last_payment_amount', 'disbursement_method',
                                               'debt_settlement_flag', 'debt_settlement_flag_date',
                                               'settlement_status', 'settlement_date', 
                                               'settlement_amount',  'settlement_percentage',
                                               'settlement_term', 'last_pymnt_d', 'last_pymnt_amnt',
                                               'last_fico_range_high', 'last_fico_range_low',
                                               'inq_last_12m', 'inq_last_6mths', 'open_rv_24m',
                                               'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                               'open_acc_6m', 'num_tl_op_past_12m',
                                               'num_tl_90g_dpd_24m', 'num_tl_30dpd',
                                               'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                               'sec_app_chargeoff_within_12_mths',
                                               'sec_app_collections_12_mths_ex_med',
                                               'zip_code', 'addr_state'], axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0,75.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,NaN,10.0,NaN,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.

## Employment Length

In [18]:
acc_clean['emp_length'].unique()

array(['10+ years', '6 years', '8 years', '2 years', '9 years', '1 year',
       '5 years', '< 1 year', '3 years', nan, '4 years', '7 years'],
      dtype=object)

In [19]:
# if employment length is 10+ years, insert 10 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='10+ years',
                                   '10 years', acc_clean['emp_length'])


# if employment length is 1 year , insert 1 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='1 year',
                                   '1 years', acc_clean['emp_length'])


# if employment length is <1, insert 0

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='< 1 year',
                                   '0 years', acc_clean['emp_length'])


# if employment length is null, insert -1

acc_clean['emp_length'] = np.where(acc_clean['emp_length'].isnull(),
                                   '-1 years', acc_clean['emp_length'])

acc_clean['emp_length']

0         10 years
1         10 years
2         10 years
3          6 years
4         10 years
            ...   
674045     5 years
674046     1 years
674047    10 years
674048     5 years
674049     3 years
Name: emp_length, Length: 674050, dtype: object

In [20]:
# removing ' years' from the column

acc_clean['emp_length'] = acc_clean['emp_length'].str.replace(' years', '', regex=False)


# Convert to numeric type

acc_clean['emp_length'] = pd.to_numeric(acc_clean['emp_length'])

acc_clean['emp_length'].unique()

array([10,  6,  8,  2,  9,  1,  5,  0,  3, -1,  4,  7], dtype=int64)

## Check if columns with dtype object should be ordinal

In [21]:
# Get columns with string values

cols_w_strings = acc_clean.loc[:, acc_clean.dtypes == object].columns

acc_clean[cols_w_strings]

,term,grade,sub_grade,emp_title,home_ownership,verification_status,issue_d,purpose,earliest_cr_line,initial_list_status,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line
0,36 months,C,C4,leadman,MORTGAGE,Not Verified,Dec-2015,debt_consolidation,Aug-2003,w,Mar-2019,Individual,NaN,NaN
1,36 months,B,B2,Vice President of Recruiting Operations,MORTGAGE,Not Verified,Dec-2015,debt_consolidation,Jun-1990,f,Mar-2019,Individual,NaN,NaN
2,36 months,B,B1,road driver,MORTGAGE,Not Verified,Dec-2015,major_purchase,Feb-1999,w,Mar-2019,Individual,NaN,NaN
3,36 months,A,A2,SERVICE MANAGER,RENT,Not Verified,Dec-2015,credit_card,Apr-2002,w,Mar-2019,Individual,NaN,NaN
4,36 months,B,B5,Vendor liaison,MORTGAGE,Not Verified,Dec-2015,credit_card,Nov-1994,w,Nov-2018,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674045,60 months,C,C1,owner,RENT,Not Verified,Oct-2016,debt_consolidation,Feb-2004,f,Mar-2019,Individual,NaN,NaN
674046,60 months,B,B4,Safety Director,OWN,Source Verified,Oct-2016,credit_card,Oct-2006,f,Oct-2017,Individual,NaN,NaN
674047,60 months,C,C3,Assistant Director,MORTGAGE,Source Verified,Oct-2016,debt_consolidation,Oct-1995,f,Jun-2017,Individual,NaN,NaN
674048,60 months,C,C4,NaN,RENT,Verified,Oct-2016,credit_card,Feb-1987,f,May-2017,Individual,NaN,NaN


In [22]:
# getting the unique values for columns that have object dtype

for col in cols_w_strings:
    print(col)
    print(acc_clean[col].unique(),'\n')


term
[' 36 months' ' 60 months'] 

grade
['C' 'B' 'A' 'E' 'D' 'F' 'G'] 

sub_grade
['C4' 'B2' 'B1' 'A2' 'B5' 'C1' 'E3' 'C3' 'A4' 'A1' 'B4' 'C5' 'D1' 'B3'
 'D3' 'C2' 'F2' 'D2' 'A5' 'E4' 'D4' 'D5' 'A3' 'E2' 'E1' 'F1' 'F3' 'E5'
 'F4' 'G5' 'F5' 'G1' 'G2' 'G3' 'G4'] 

emp_title
['leadman' 'Vice President of Recruiting Operations' 'road driver' ...
 'Construction management ' 'RN, Med-Surg Manager' 'STAFF CHAPLAIN'] 

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE' 'OTHER'] 

verification_status
['Not Verified' 'Verified' 'Source Verified'] 

issue_d
['Dec-2015' 'Nov-2015' 'Oct-2015' 'Sep-2015' 'Aug-2015' 'Jul-2015'
 'Jun-2015' 'May-2015' 'Apr-2015' 'Mar-2015' 'Feb-2015' 'Jan-2015'
 'Mar-2018' 'Feb-2018' 'Jan-2018' 'Sep-2017' 'Aug-2017' 'Jul-2017'
 'Jun-2016' 'May-2016' 'Apr-2016' 'Sep-2018' 'Aug-2018' 'Jul-2018'
 'Jun-2017' 'May-2017' 'Apr-2017' 'Mar-2016' 'Feb-2016' 'Jan-2016'
 'Dec-2014' 'Nov-2014' 'Oct-2014' 'Sep-2014' 'Aug-2014' 'Jul-2014'
 'Jun-2014' 'May-2014' 'Apr-2014' 'Mar-201

 'Mar-1961'] 

initial_list_status
['w' 'f'] 

last_credit_pull_d
['Mar-2019' 'Nov-2018' 'Jun-2016' 'Aug-2018' 'Jan-2019' 'Jun-2017'
 'Jul-2016' 'Feb-2019' 'Dec-2018' 'Feb-2018' 'Apr-2018' 'Aug-2017'
 'Sep-2018' 'Mar-2016' 'Nov-2017' 'Jun-2018' 'May-2018' 'Dec-2016'
 'Jul-2017' 'Mar-2018' 'Jan-2018' 'Oct-2018' 'Dec-2017' 'May-2017'
 'Mar-2017' 'Jan-2017' 'Jul-2018' 'Aug-2016' 'Feb-2016' 'Apr-2017'
 'Sep-2017' 'Apr-2016' 'Oct-2017' 'Oct-2016' 'Feb-2017' 'Nov-2016'
 'Jan-2016' 'Sep-2016' 'May-2016' 'Dec-2015' 'Nov-2015' nan 'Oct-2015'
 'Sep-2015' 'Aug-2015' 'Jul-2015' 'Jun-2015' 'May-2015' 'Apr-2015'
 'Mar-2015' 'Feb-2015' 'Jan-2015' 'Dec-2014' 'Nov-2014' 'Oct-2014'
 'Sep-2014' 'Aug-2014' 'Jul-2014' 'Jun-2014' 'May-2014' 'Apr-2014'
 'Mar-2014' 'Feb-2014' 'Jan-2014' 'Jun-2012' 'Dec-2012' 'Feb-2012'
 'May-2013' 'Jan-2012' 'May-2012' 'Jun-2013' 'Sep-2013' 'Feb-2013'
 'Apr-2013' 'Dec-2013' 'Aug-2012' 'Oct-2013' 'Nov-2013' 'Jul-2012'
 'Jul-2013' 'Nov-2012' 'Mar-2013' 'Sep-2012' 'Aug-2013' 'Ap

# Grade

In [23]:
# replacing letter grades with numbers

# get sorted list of grades
grade_list = sorted(list(acc_clean['grade'].unique()))

i = 1

for grades in grade_list:
    acc_clean.loc[acc_clean['grade']==grades, 'grade'] = i
    i+=1

acc_clean['grade'] = acc_clean['grade'].astype(int)
acc_clean['grade'].unique()

array([3, 2, 1, 5, 4, 6, 7])

In [24]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['sub_grade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['sub_grade']==subgrades, 'sub_grade'] = i
    i+=1

acc_clean['sub_grade'] = acc_clean['sub_grade'].astype(int)
acc_clean['sub_grade'].unique()

array([14,  7,  6,  2, 10, 11, 23, 13,  4,  1,  9, 15, 16,  8, 18, 12, 27,
       17,  5, 24, 19, 20,  3, 22, 21, 26, 28, 25, 29, 35, 30, 31, 32, 33,
       34])

### Earliest Credit Line

In [25]:
# converting the column to datetime format

acc_clean['earliest_cr_line_dt'] = pd.DatetimeIndex(acc_clean['earliest_cr_line'])
acc_clean['earliest_cr_line_dt']

0        2003-08-01
1        1990-06-01
2        1999-02-01
3        2002-04-01
4        1994-11-01
            ...    
674045   2004-02-01
674046   2006-10-01
674047   1995-10-01
674048   1987-02-01
674049   2011-06-01
Name: earliest_cr_line_dt, Length: 674050, dtype: datetime64[ns]

In [26]:
# creating a new column that will show the number of months that credit has existed for customer

acc_clean['duration_of_credit_months'] =  (pd.DatetimeIndex(acc_clean['issue_d']).year -\
                                               pd.DatetimeIndex(acc_clean['earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_clean['issue_d']).month - pd.DatetimeIndex(acc_clean['earliest_cr_line']).month)

acc_clean[['duration_of_credit_months']]

,duration_of_credit_months
0,148.0
1,306.0
2,202.0
3,164.0
4,253.0
...,...
674045,152.0
674046,120.0
674047,252.0
674048,356.0


In [27]:
# dropping the columns that helped create the new column

acc_clean = acc_clean.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)

### Last Credit Pulled Date

In [28]:
# dropping the column bc it is used post-funding

acc_clean = acc_clean.drop(['last_credit_pull_d'], axis=1)

# Fico Score

In [29]:
acc_clean['fico_score'] = (acc_clean['fico_range_high'] + acc_clean['fico_range_low'])/2

In [30]:
acc_clean['sec_app_fico_score'] = (acc_clean['sec_app_fico_range_high'] +\
                                   acc_clean['sec_app_fico_range_low'])/2

In [31]:
acc_clean = acc_clean.drop(['fico_range_high', 'fico_range_low',
                                    'sec_app_fico_range_high',
                                    'sec_app_fico_range_low'], axis=1)

### Verification status

In [32]:
acc_clean['verification_status'].unique()

array(['Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [33]:
# if the income is verified, change to 1. If not verified, change to 0

acc_clean.loc[((acc_clean['verification_status']=='Source Verified') |\
                  (acc_clean['verification_status']=='Verified')), 'verification_status'] = 1

acc_clean.loc[acc_clean['verification_status']=='Not Verified', 'verification_status'] = 0

# change dtype to int
acc_clean['verification_status'] = acc_clean['verification_status'].astype(int)

acc_clean['verification_status']

0         0
1         0
2         0
3         0
4         0
         ..
674045    0
674046    1
674047    1
674048    1
674049    1
Name: verification_status, Length: 674050, dtype: int32

In [34]:
acc_clean['application_type'].value_counts()

Individual    661254
Joint App      12796
Name: application_type, dtype: int64

In [35]:
list_sec_app = list(acc_clean.filter(like='sec_app').columns)
list_sec_app.append('annual_inc_joint')
list_sec_app

['sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_mths_since_last_major_derog',
 'sec_app_fico_score',
 'annual_inc_joint']

In [36]:
acc_clean[(acc_clean['sec_app_fico_score'].notnull())|
              (acc_clean['sec_app_earliest_cr_line'].notnull())|
              (acc_clean['sec_app_inq_last_6mths'].notnull())|
              (acc_clean['sec_app_mort_acc'].notnull())|
              (acc_clean['sec_app_open_acc'].notnull())|
              (acc_clean['sec_app_revol_util'].notnull())|
              (acc_clean['sec_app_open_act_il'].notnull())|
              (acc_clean['sec_app_num_rev_accts'].notnull())|
              (acc_clean['sec_app_mths_since_last_major_derog'].notnull())|
              (acc_clean['annual_inc_joint'].notnull())]


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score
157,15000.0,15000.0,15000.0,36 months,11.99,498.15,3,11,Lead Tech,1,MORTGAGE,65000.0,1,Dec-2015,debt_consolidation,17.56,0.0,54.0,NaN,9.0,0.0,13028.0,88.3,16.0,w,15183.460000,0.0,NaN,1.0,Joint App,77000.0,15.40,Not Verified,0.0,0.0,231860.0,0.0,30.0,22525.0,74.0,2839.0,78.0,14800.0,3.0,1.0,2.0,25762.0,295.0,94.2,0.0,0.0,99.0,136.0,13.0,13.0,1.0,28.0,NaN,11.0,54.0,2.0,7.0,2.0,3.0,4.0,7.0,11.0,7.0,9.0,92.3,100.0,0.0,0.0,251645.0,35553.0,5100.0,30649.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,667.0,NaN
186,24000.0,24000.0,24000.0,60 months,23.13,678.37,6,27,Coordinator,6,MORTGAGE,43435.0,1,Dec-2015,debt_consolidation,35.70,0.0,NaN,84.0,13.0,1.0,12508.0,73.1,22.0,w,27488.251245,0.0,NaN,1.0,Joint App,113314.0,25.44,Not Verified,0.0,156.0,182215.0,6.0,10.0,48582.0,75.0,5690.0,74.0,17100.0,1.0,2.0,6.0,14017.0,357.0,96.8,0.0,0.0,52.0,209.0,13.0,10.0,1.0,26.0,NaN,12.0,NaN,2.0,5.0,2.0,4.0,9.0,6.0,12.0,5.0,13.0,100.0,100.0,1.0,0.0,208064.0,61090.0,11000.0,61739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0,682.0,NaN
193,24000.0,24000.0,24000.0,60 months,13.44,551.50,3,13,Transportation Manager,1,MORTGAGE,96000.0,1,Dec-2015,credit_card,22.70,0.0,55.0,63.0,13.0,1.0,19403.0,53.3,24.0,w,30785.080592,0.0,55.0,1.0,Joint App,117000.0,22.76,Not Verified,0.0,0.0,220879.0,4.0,10.0,43145.0,70.0,6210.0,64.0,36400.0,2.0,0.0,7.0,16991.0,6328.0,60.9,0.0,0.0,158.0,71.0,11.0,10.0,2.0,19.0,NaN,6.0,55.0,2.0,5.0,3.0,4.0,8.0,8.0,13.0,5.0,13.0,95.7,66.7,0.0,0.0,258301.0,62548.0,16200.0,61251.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174.0,677.0,NaN
343,35000.0,35000.0,35000.0,60 months,17.97,888.20,4,19,IT Analyst,10,MORTGAGE,150000.0,0,Dec-2015,debt_consolidation,25.11,1.0,20.0,NaN,27.0,0.0,39913.0,85.3,45.0,w,43193.305750,0.0,25.0,1.0,Joint App,192000.0,25.28,Not Verified,0.0,433.0,603735.0,5.0,11.0,139765.0,74.0,13130.0,78.0,46800.0,3.0,8.0,5.0,22361.0,3761.0,89.4,0.0,0.0,165.0,404.0,0.0,0.0,7.0,29.0,20.0,0.0,20.0,9.0,15.0,10.0,15.0,9.0,18.0,27.0,16.0,27.0,84.4,80.0,0.0,0.0,733890.0,180363.0,35600.0,166148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.0,672.0,NaN
887,10500.0,10500.0,10500.0,36 months,9.80,337.83,2,8,Supervisor,0,MORTGAGE,65000.0,1,Dec-2015,debt_consolidation,21.49,1.0,10.0,NaN,29.0,0.0,11156.0,83.9,43.0,w,12150.121933,0.0,65.0,1.0,Joint App,92000.0,15.85,Not Verified,0.0,0.0,350643.0,24.0,1.0,167875.0,96.0,4970.0,95.0,13300.0,0.0,1.0,7.0,12091.0,2144.0,83.9,0.0,0.0,127.0,287.0,17.0,1.0,1.0,17.0,10.0,2.0,10.0,4.0,4.0,4.0,6.0,36.0,4.0,6.0,4.0,29.0,93.0,50.0,0.0,0.0,373179.0,179031.0,13300.0,184122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.0,667.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,

In [37]:
acc_clean.loc[acc_clean['application_type']=='Joint App', 'application_type'] = 2
acc_clean.loc[acc_clean['application_type']=='Individual', 'application_type'] = 1

acc_clean['application_type'] = acc_clean['application_type'].astype(int)

In [38]:
acc_clean['verification_status_joint'].unique()

array([nan, 'Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [39]:
'''
for verification_status_joint where there are values in cols that start with
sec_app, but verification_status_joint is null
'''

# add in 'not verified' where applicable

acc_clean.loc[((acc_clean['verification_status_joint'].notnull()) &\
                   (acc_clean['application_type']==1)),
                  'verification_status_joint'] = 'Not Verified'

In [40]:
acc_clean[((acc_clean['verification_status_joint'].isnull()) &\
                  (acc_clean['application_type']==2))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score
150383,14500.0,14500.0,14500.0,36 months,5.31,436.60,1,1,Software Developer,2,MORTGAGE,74000.0,1,Mar-2018,debt_consolidation,9.22,0.0,NaN,NaN,5.0,0.0,16257.0,45.2,7.0,w,15116.591168,0.0,NaN,1.0,2,136000.0,9.46,NaN,0.0,0.0,176952.0,1.0,6.0,3486.0,48.0,14095.0,46.0,36000.0,1.0,0.0,4.0,35390.0,18092.0,45.2,0.0,0.0,79.0,261.0,3.0,3.0,1.0,10.0,NaN,0.0,NaN,2.0,3.0,2.0,2.0,2.0,3.0,4.0,3.0,5.0,100.0,0.0,0.0,0.0,200825.0,19743.0,33000.0,7200.0,38271.0,Sep-2010,1.0,1.0,13.0,74.4,2.0,11.0,NaN,261.0,742.0,622.0
150408,35000.0,35000.0,35000.0,60 months,18.45,897.37,4,17,Office Specialist III,2,MORTGAGE,65000.0,1,Mar-2018,debt_consolidation,17.43,0.0,NaN,NaN,10.0,0.0,32693.0,74.5,12.0,w,40066.441011,0.0,NaN,1.0,2,148000.0,13.64,NaN,0.0,0.0,403920.0,0.0,NaN,0.0,NaN,16322.0,75.0,43900.0,1.0,0.0,5.0,40392.0,5384.0,85.2,0.0,0.0,NaN,76.0,11.0,1.0,3.0,12.0,NaN,0.0,NaN,6.0,9.0,6.0,6.0,0.0,9.0,9.0,9.0,10.0,100.0,66.7,0.0,0.0,415127.0,32693.0,36400.0,0.0,34704.0,Nov-2007,0.0,3.0,9.0,77.7,1.0,9.0,NaN,76.0,672.0,677.0
150711,23000.0,23000.0,23000.0,60 months,16.01,559.44,3,15,NaN,-1,MORTGAGE,0.0,1,Mar-2018,debt_consolidation,NaN,0.0,NaN,93.0,33.0,1.0,27044.0,64.1,46.0,w,24635.767153,0.0,NaN,1.0,2,99000.0,23.33,NaN,0.0,212.0,363821.0,20.0,2.0,109476.0,93.0,7611.0,85.0,42200.0,2.0,1.0,11.0,11025.0,9838.0,69.6,0.0,0.0,144.0,131.0,8.0,2.0,1.0,8.0,NaN,2.0,NaN,6.0,10.0,7.0,8.0,32.0,12.0,13.0,10.0,33.0,100.0,71.4,1.0,0.0,409719.0,136520.0,32400.0,117678.0,41582.0,Mar-2006,0.0,1.0,25.0,78.8,16.0,11.0,NaN,144.0,712.0,682.0
151304,21000.0,21000.0,21000.0,60 months,10.42,450.55,2,8,Deputy Sheriff,6,MORTGAGE,60000.0,0,Mar-2018,debt_consolidation,30.92,0.0,NaN,NaN,11.0,0.0,16864.0,66.9,19.0,w,21853.282141,0.0,NaN,1.0,2,100000.0,22.47,NaN,0.0,0.0,201975.0,3.0,22.0,42498.0,71.0,10039.0,69.0,25200.0,3.0,2.0,2.0,18361.0,5936.0,74.0,0.0,0.0,80.0,72.0,21.0,21.0,1.0,21.0,NaN,15.0,NaN,5.0,5.0,6.0,8.0,8.0,7.0,10.0,5.0,11.0,94.7,16.7,0.0,0.0,240520.0,59362.0,22800.0,66074.0,27385.0,Nov-2010,0.0,0.0,19.0,87.1,15.0,5.0,70.0,81.0,707.0,652.0
151361,10000.0,10000.0,10000.0,36 months,5.31,301.11,1,1,Commercial Lender,0,MORTGAGE,125000.0,0,Mar-2018,credit_card,7.63,0.0,NaN,NaN,6.0,0.0,10152.0,28.1,19.0,w,10352.759836,0.0,NaN,1.0,2,195000.0,5.17,NaN,0.0,0.0,527602.0,1.0,24.0,15239.0,53.0,9838.0,39.0,36100.0,1.0,2.0,5.0,87934.0,25948.0,28.1,0.0,0.0,98.0,177.0,6.0,6.0,5.0,6.0,NaN,12.0,NaN,2.0,2.0,3.0,7.0,6.0,3.0,8.0,2.0,6.0,100.0,0.0,0.0,0.0,573426.0,25391.0,36100.0,28861.0,11266.0,Feb-2002,0.0,5.0,7.0,21.5,1.0,7.0,NaN,177.0,772.0,712.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [41]:
acc_clean.loc[((acc_clean['verification_status_joint'].isnull()) &\
                   (acc_clean['application_type']==2)),
                  'verification_status_joint'] = 'Not Verified'

In [42]:
acc_clean[((acc_clean['verification_status_joint'].notnull()) &\
                  (acc_clean['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score


In [43]:
acc_clean[((acc_clean['verification_status_joint'].isnull()) &\
               (acc_clean['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,3,14,leadman,10,MORTGAGE,55000.0,0,Dec-2015,debt_consolidation,5.91,0.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0,677.0,NaN
1,20000.0,20000.0,20000.0,36 months,9.17,637.58,2,7,Vice President of Recruiting Operations,10,MORTGAGE,180000.0,0,Dec-2015,debt_consolidation,14.67,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306.0,682.0,NaN
2,20000.0,20000.0,20000.0,36 months,8.49,631.26,2,6,road driver,10,MORTGAGE,85000.0,0,Dec-2015,major_purchase,17.61,1.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,707.0,NaN
3,10000.0,10000.0,10000.0,36 months,6.49,306.45,1,2,SERVICE MANAGER,6,RENT,85000.0,0,Dec-2015,credit_card,13.07,0.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164.0,687.0,NaN
4,8000.0,8000.0,8000.0,36 months,11.48,263.74,2,10,Vendor liaison,10,MORTGAGE,42000.0,0,Dec-2015,credit_card,34.80,0.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,NaN,NaN,NaN,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,NaN,10.0,NaN,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253.0,702.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [44]:
acc_clean.loc[((acc_clean['verification_status_joint'].isnull()) &\
                   (acc_clean['application_type']==1)),
                  'verification_status_joint'] = 'Individual'

In [45]:
# if the income is verified, change to 1. If not verified, change to 0

acc_clean.loc[((acc_clean['verification_status_joint']=='Source Verified') |\
                  (acc_clean['verification_status_joint']=='Verified')),
                  'verification_status_joint'] = 1

acc_clean.loc[acc_clean['verification_status_joint']=='Individual',
                  'verification_status_joint'] = -1

acc_clean.loc[acc_clean['verification_status_joint']=='Not Verified',
                  'verification_status_joint'] = 0

In [46]:
acc_clean['verification_status_joint'] = acc_clean['verification_status_joint'].astype(int)

### Joint Earliest Credit Line

In [47]:
# converting column to datetime

acc_clean['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line'])
acc_clean['sec_app_earliest_cr_line_dt']

0        NaT
1        NaT
2        NaT
3        NaT
4        NaT
          ..
674045   NaT
674046   NaT
674047   NaT
674048   NaT
674049   NaT
Name: sec_app_earliest_cr_line_dt, Length: 674050, dtype: datetime64[ns]

In [48]:
# checking how many non-null values there are

acc_clean[acc_clean['sec_app_earliest_cr_line'].notnull()]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_earliest_cr_line_dt
149593,1000.0,1000.0,1000.0,36 months,17.47,35.89,4,16,Lead Sanitation Operator,2,RENT,51000.0,1,Mar-2018,credit_card,10.56,0.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,1032.903619,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,3.0,17.0,22705.0,95.0,2.0,86.0,2500.0,4.0,0.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,62.0,120.0,707.0,637.0,2007-02-01
149599,7500.0,7500.0,7475.0,36 months,7.34,232.75,1,4,"Vice President, Client Services",10,OWN,300000.0,1,Mar-2018,other,8.91,0.0,43.0,NaN,9.0,0.0,12750.0,34.6,23.0,f,7665.955092,0.0,68.0,1.0,2,500000.0,7.66,1,0.0,0.0,423667.0,2.0,6.0,54103.0,86.0,5143.0,57.0,36800.0,1.0,0.0,4.0,52958.0,23050.0,35.6,0.0,0.0,186.0,155.0,1.0,1.0,3.0,1.0,54.0,5.0,54.0,3.0,3.0,5.0,7.0,11.0,6.0,9.0,3.0,9.0,81.8,0.0,0.0,0.0,478415.0,66853.0,35800.0,73279.0,13308.0,Feb-2009,1.0,0.0,6.0,93.0,5.0,3.0,49.0,186.0,722.0,602.0,2009-02-01
149606,5750.0,5750.0,5725.0,36 months,13.58,195.36,3,12,Housekeeing,1,RENT,1200.0,1,Mar-2018,small_business,63.00,0.0,NaN,NaN,12.0,0.0,2408.0,12.4,13.0,f,6346.533647,0.0,NaN,1.0,2,18000.0,4.47,1,0.0,0.0,2408.0,0.0,NaN,0.0,NaN,1875.0,12.0,19400.0,0.0,0.0,11.0,201.0,6925.0,21.3,0.0,0.0,NaN,37.0,4.0,4.0,0.0,4.0,NaN,15.0,NaN,1.0,2.0,4.0,4.0,0.0,12.0,13.0,2.0,12.0,100.0,0.0,0.0,0.0,19400.0,2408.0,8800.0,0.0,2412.0,Jul-2015,0.0,0.0,7.0,0.0,0.0,8.0,NaN,37.0,727.0,667.0,2015-07-01
149623,32000.0,32000.0,32000.0,36 months,18.45,1164.12,4,17,Tech Support,5,RENT,78000.0,1,Mar-2018,debt_consolidation,44.69,0.0,NaN,NaN,16.0,0.0,53179.0,70.0,26.0,w,36456.075686,0.0,NaN,1.0,2,157000.0,26.35,1,0.0,0.0,98044.0,2.0,15.0,44865.0,72.0,14250.0,71.0,76000.0,6.0,0.0,10.0,6128.0,13865.0,77.6,0.0,0.0,120.0,175.0,6.0,6.0,0.0,6.0,NaN,15.0,NaN,9.0,12.0,10.0,11.0,9.0,14.0,17.0,12.0,16.0,100.0,50.0,0.0,0.0,138106.0,98044.0,62000.0,62106.0,70727.0,Aug-1999,2.0,0.0,12.0,60.3,1.0,12.0,NaN,175.0,662.0,627.0,1999-08-01
149627,30000.0,30000.0,30000.0,36 months,6.07,913.61,1,2,Community Liaison,3,MORTGAGE,150000.0,1,Mar-2018,debt_consolidation,23.40,0.0,NaN,NaN,15.0,0.0,20979.0,20.0,39.0,w,30406.296711,0.0,NaN,1.0,2,230000.0,20.05,1,0.0,0.0,799866.0,4.0,12.0,116462.0,83.0,20786.0,48.0,104800.0,6.0,2.0,11.0,57133.0,81821.0,20.4,0.0,0.0,142.0,221.0,7.0,7.0,14.0,8.0,NaN,2.0,NaN,2.0,2.0,7.0,11.0,8.0,8.0,17.0,2.0,15.0,100.0,0.0,0.0,0.0,967222.0,137441.0,102800.0,138492.0,137337.0,Jan-2001,1.0,14.0,11.0,13.9,3.0,10.0,NaN,221.0,807.0,802.0,2001-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...

In [49]:
# confirming that we can use 0 as a plausible value for null values

acc_clean[acc_clean['sec_app_earliest_cr_line']<=acc_clean['issue_d']]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_earliest_cr_line_dt
149593,1000.0,1000.0,1000.0,36 months,17.47,35.89,4,16,Lead Sanitation Operator,2,RENT,51000.0,1,Mar-2018,credit_card,10.56,0.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,1032.903619,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,3.0,17.0,22705.0,95.0,2.0,86.0,2500.0,4.0,0.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,62.0,120.0,707.0,637.0,2007-02-01
149599,7500.0,7500.0,7475.0,36 months,7.34,232.75,1,4,"Vice President, Client Services",10,OWN,300000.0,1,Mar-2018,other,8.91,0.0,43.0,NaN,9.0,0.0,12750.0,34.6,23.0,f,7665.955092,0.0,68.0,1.0,2,500000.0,7.66,1,0.0,0.0,423667.0,2.0,6.0,54103.0,86.0,5143.0,57.0,36800.0,1.0,0.0,4.0,52958.0,23050.0,35.6,0.0,0.0,186.0,155.0,1.0,1.0,3.0,1.0,54.0,5.0,54.0,3.0,3.0,5.0,7.0,11.0,6.0,9.0,3.0,9.0,81.8,0.0,0.0,0.0,478415.0,66853.0,35800.0,73279.0,13308.0,Feb-2009,1.0,0.0,6.0,93.0,5.0,3.0,49.0,186.0,722.0,602.0,2009-02-01
149606,5750.0,5750.0,5725.0,36 months,13.58,195.36,3,12,Housekeeing,1,RENT,1200.0,1,Mar-2018,small_business,63.00,0.0,NaN,NaN,12.0,0.0,2408.0,12.4,13.0,f,6346.533647,0.0,NaN,1.0,2,18000.0,4.47,1,0.0,0.0,2408.0,0.0,NaN,0.0,NaN,1875.0,12.0,19400.0,0.0,0.0,11.0,201.0,6925.0,21.3,0.0,0.0,NaN,37.0,4.0,4.0,0.0,4.0,NaN,15.0,NaN,1.0,2.0,4.0,4.0,0.0,12.0,13.0,2.0,12.0,100.0,0.0,0.0,0.0,19400.0,2408.0,8800.0,0.0,2412.0,Jul-2015,0.0,0.0,7.0,0.0,0.0,8.0,NaN,37.0,727.0,667.0,2015-07-01
149623,32000.0,32000.0,32000.0,36 months,18.45,1164.12,4,17,Tech Support,5,RENT,78000.0,1,Mar-2018,debt_consolidation,44.69,0.0,NaN,NaN,16.0,0.0,53179.0,70.0,26.0,w,36456.075686,0.0,NaN,1.0,2,157000.0,26.35,1,0.0,0.0,98044.0,2.0,15.0,44865.0,72.0,14250.0,71.0,76000.0,6.0,0.0,10.0,6128.0,13865.0,77.6,0.0,0.0,120.0,175.0,6.0,6.0,0.0,6.0,NaN,15.0,NaN,9.0,12.0,10.0,11.0,9.0,14.0,17.0,12.0,16.0,100.0,50.0,0.0,0.0,138106.0,98044.0,62000.0,62106.0,70727.0,Aug-1999,2.0,0.0,12.0,60.3,1.0,12.0,NaN,175.0,662.0,627.0,1999-08-01
149627,30000.0,30000.0,30000.0,36 months,6.07,913.61,1,2,Community Liaison,3,MORTGAGE,150000.0,1,Mar-2018,debt_consolidation,23.40,0.0,NaN,NaN,15.0,0.0,20979.0,20.0,39.0,w,30406.296711,0.0,NaN,1.0,2,230000.0,20.05,1,0.0,0.0,799866.0,4.0,12.0,116462.0,83.0,20786.0,48.0,104800.0,6.0,2.0,11.0,57133.0,81821.0,20.4,0.0,0.0,142.0,221.0,7.0,7.0,14.0,8.0,NaN,2.0,NaN,2.0,2.0,7.0,11.0,8.0,8.0,17.0,2.0,15.0,100.0,0.0,0.0,0.0,967222.0,137441.0,102800.0,138492.0,137337.0,Jan-2001,1.0,14.0,11.0,13.9,3.0,10.0,NaN,221.0,807.0,802.0,2001-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...

In [50]:
# creating a new column that will show the number of months that credit has existed for joint customer

acc_clean['sec_app_duration_of_credit_months'] = (pd.DatetimeIndex(acc_clean['issue_d']).year\
                                                      - pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_clean['issue_d']).month - pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line']).month)

# converting null values to -1
acc_clean.loc[acc_clean['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

acc_clean['sec_app_duration_of_credit_months']

0        -1.0
1        -1.0
2        -1.0
3        -1.0
4        -1.0
         ... 
674045   -1.0
674046   -1.0
674047   -1.0
674048   -1.0
674049   -1.0
Name: sec_app_duration_of_credit_months, Length: 674050, dtype: float64

In [51]:
acc_clean = acc_clean.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
#                                     'issue_d', 
                                    'sec_app_earliest_cr_line',
                                    'sec_app_earliest_cr_line_dt'], axis=1)

## Check the max values of columns starting with 'mths_since_'

In [52]:
cols_w_mths_since = [col for col in acc_clean.columns if col.startswith('mths_since_')]
acc_clean[cols_w_mths_since]

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq
0,30.0,NaN,30.0,21.0,4.0,69.0,4.0,69.0
1,49.0,NaN,NaN,18.0,12.0,NaN,10.0,NaN
2,3.0,NaN,3.0,13.0,32.0,NaN,8.0,NaN
3,NaN,106.0,NaN,35.0,4.0,NaN,1.0,NaN
4,75.0,NaN,75.0,10.0,50.0,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...
674045,17.0,NaN,NaN,13.0,29.0,17.0,5.0,17.0
674046,NaN,64.0,NaN,11.0,45.0,NaN,2.0,NaN
674047,2.0,NaN,NaN,15.0,3.0,NaN,3.0,NaN
674048,NaN,NaN,NaN,6.0,38.0,NaN,6.0,NaN


In [53]:
acc_clean[cols_w_mths_since].max()

mths_since_last_delinq            188.0
mths_since_last_record            124.0
mths_since_last_major_derog       197.0
mths_since_rcnt_il                407.0
mths_since_recent_bc              639.0
mths_since_recent_bc_dlq          189.0
mths_since_recent_inq              25.0
mths_since_recent_revol_delinq    197.0
dtype: float64

In [54]:
for col in cols_w_mths_since:
    acc_clean.loc[acc_clean[col].isnull(), col] = 10000

In [55]:
acc_clean[cols_w_mths_since].max()

mths_since_last_delinq            10000.0
mths_since_last_record            10000.0
mths_since_last_major_derog       10000.0
mths_since_rcnt_il                10000.0
mths_since_recent_bc              10000.0
mths_since_recent_bc_dlq          10000.0
mths_since_recent_inq             10000.0
mths_since_recent_revol_delinq    10000.0
dtype: float64

## Get cols with null values

In [56]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

,emp_title,annual_inc,dti,delinq_2yrs,open_acc,pub_rec,revol_util,total_acc,collections_12_mths_ex_med,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,sec_app_fico_score
0,leadman,55000.0,5.91,0.0,7.0,0.0,29.7,13.0,0.0,NaN,NaN,0.0,722.0,144904.0,2.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0,NaN
1,Vice President of Recruiting Operations,180000.0,14.67,0.0,12.0,0.0,84.5,27.0,0.0,NaN,NaN,0.0,0.0,360358.0,2.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306.0,NaN
2,road driver,85000.0,17.61,1.0,8.0,0.0,5.7,15.0,0.0,NaN,NaN,0.0,0.0,141601.0,3.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,NaN
3,SERVICE MANAGER,85000.0,13.07,0.0,14.0,1.0,34.5,23.0,0.0,NaN,NaN,0.0,8341.0,27957.0,1.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164.0,NaN
4,Vendor liaison,42000.0,34.80,0.0,8.0,0.0,39.1,18.0,0.0,NaN,NaN,0.0,0.0,199696.0,2.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674045,owner,62000.0,14.46,1.0,12.0,0.0,34.1,22.0,0.0,NaN,NaN,0.0,0.0,17516.0,1.0,10736.0,72.0,3245.0,50.0,19900.0,1.0,0.0,3.0,1592.0,11820.0,36.5,0.0,0.0,14.0,151.0,21.0,13.0,0.0,7.0,7.0,9.0,14.0,2.0,11.0,20.0,7.0,12.0,90.9,25.0,0.0,0.0,34900.0,17516.0,18600.0,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0,NaN
674046,Safety Director,49400.0,13.85,0.0,18.0,1.0,35.2,28.0,0.0,NaN,NaN,0.0,0.0,21772.0,1.0,15993.0,88.0,2151.0,63.0,16400.0,0.0,1.0,12.0,1361.0,757.0,76.3,0.0,0.0,110.0,119.0,6.0,6.0,0.0,2.0,8.0,2.0,8.0,4.0,17.0,24.0,8.0,18.0,100.0,50.0,1.0,0.0,34605.0,21772.0,3200.0,18205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN
674047,Assistant Director,125000.0,29.89,1.0,15.0,0.0,41.6,38.0,0.0,NaN,NaN,1.0,0.0,436060.0,4.0,93273.0,56.0,22031.0,44.0,107800.0,0.0,4.0,6.0,29071.0,53336.0,43.6,0.0,2141.0,124.0,251.0,3.0,3.0,7.0,6.0,7.0,8.0,14.0,14.0,10.0,17.0,7.0,14.0,97.3,25.0,0.0,0.0,557391.0,138125.0,94600.0,124725.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0,NaN
674048,NaN,95000.0,21.50,0.0,11.0,0.0,79.8,24.0,0.0,NaN,NaN,0.0,0.0,481490.0,2.0,81554.0,75.0,16390.0,78.0,69800.0,1.0,4.0,3.0,43772.0,3676.0,91.5,0.0,0.0,82.0,355.0,5.0,5.0,6.0,3.0,5.0,4.0,6.0,5.0,7.0,13.0,5.0,11.0,100.0,75.0,0.0,0.0,529965.0,137277.0,43500.0,90665.0,NaN,NaN,NaN,NaN,NaN,N

In [57]:
cols_w_null

['emp_title',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'open_acc',
 'pub_rec',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_act_il',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'revol_bal_joint',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol

## Fill in null values

In [58]:
# creates a list of the index where the column == 0

list(acc_clean[acc_clean['emp_length']==0].index)

[15,
 27,
 34,
 47,
 49,
 55,
 63,
 65,
 115,
 137,
 142,
 144,
 150,
 152,
 171,
 177,
 181,
 187,
 188,
 208,
 220,
 225,
 235,
 260,
 270,
 284,
 294,
 307,
 353,
 362,
 365,
 372,
 378,
 420,
 422,
 431,
 443,
 446,
 449,
 456,
 458,
 461,
 467,
 468,
 471,
 472,
 483,
 485,
 500,
 511,
 526,
 580,
 587,
 603,
 613,
 615,
 619,
 626,
 628,
 632,
 657,
 667,
 668,
 697,
 712,
 720,
 722,
 728,
 732,
 749,
 750,
 782,
 799,
 808,
 823,
 843,
 859,
 861,
 884,
 887,
 890,
 906,
 909,
 916,
 926,
 943,
 953,
 963,
 967,
 975,
 981,
 983,
 1008,
 1016,
 1020,
 1025,
 1029,
 1043,
 1048,
 1053,
 1079,
 1080,
 1092,
 1094,
 1101,
 1107,
 1110,
 1120,
 1123,
 1128,
 1133,
 1135,
 1137,
 1142,
 1145,
 1162,
 1202,
 1206,
 1212,
 1213,
 1225,
 1229,
 1233,
 1240,
 1250,
 1261,
 1287,
 1294,
 1296,
 1311,
 1321,
 1322,
 1332,
 1335,
 1351,
 1364,
 1368,
 1370,
 1379,
 1406,
 1412,
 1414,
 1415,
 1417,
 1443,
 1452,
 1454,
 1475,
 1476,
 1483,
 1486,
 1525,
 1544,
 1552,
 1579,
 1593,
 1606,
 

In [59]:
# getting the columns where there are 0s

cols_w_0s_and_null = []

for col in cols_w_null:
    
    # if the list is not empty, then there are values == 0
    if list(acc_clean[acc_clean[col]==0].index)!=[]:
        cols_w_0s_and_null.append(col)
    
    for col_0 in cols_w_0s_and_null:
        acc_clean.loc[acc_clean[col_0].isnull(), col_0] = -1
    
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,3,14,leadman,10,MORTGAGE,55000.0,0,Dec-2015,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,NaN,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,148.0,677.0,NaN,-1.0
1,20000.0,20000.0,20000.0,36 months,9.17,637.58,2,7,Vice President of Recruiting Operations,10,MORTGAGE,180000.0,0,Dec-2015,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,NaN,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,306.0,682.0,NaN,-1.0
2,20000.0,20000.0,20000.0,36 months,8.49,631.26,2,6,road driver,10,MORTGAGE,85000.0,0,Dec-2015,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,NaN,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,202.0,707.0,NaN,-1.0
3,10000.0,10000.0,10000.0,36 months,6.49,306.45,1,2,SERVICE MANAGER,6,RENT,85000.0,0,Dec-2015,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,NaN,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,164.0,687.0,NaN,-1.0
4,8000.0,8000.0,8000.0,36 months,11.48,263.74,2,10,Vendor liaison,10,MORTGAGE,42000.0,0,Dec-2015,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,NaN,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,253.0,702.0,NaN,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [60]:
# rerun to get new cols w null values

cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

,emp_title,total_acc,annual_inc_joint,mo_sin_old_rev_tl_op,duration_of_credit_months,sec_app_fico_score
0,leadman,13.0,NaN,128.0,148.0,NaN
1,Vice President of Recruiting Operations,27.0,NaN,306.0,306.0,NaN
2,road driver,15.0,NaN,55.0,202.0,NaN
3,SERVICE MANAGER,23.0,NaN,129.0,164.0,NaN
4,Vendor liaison,18.0,NaN,253.0,253.0,NaN
...,...,...,...,...,...,...
674045,owner,22.0,NaN,151.0,152.0,NaN
674046,Safety Director,28.0,NaN,119.0,120.0,NaN
674047,Assistant Director,38.0,NaN,251.0,252.0,NaN
674048,NaN,24.0,NaN,355.0,356.0,NaN


In [61]:
acc_clean.loc[acc_clean['emp_title'].isnull(), 'emp_title'] = 'Unknown'

### Employment length

In [62]:
emplength_mean = acc_clean['emp_length'].mean()

In [63]:
acc_clean.loc[(acc_clean['emp_length'].isnull()) &\
                  (acc_clean['emp_title'].notnull()), 'emp_length'] = emplength_mean

In [64]:
acc_clean.loc[(acc_clean['emp_length'].isnull()) &\
                  (acc_clean['emp_title'].isnull()), 'emp_length'] = -1

### Employment Title

In [65]:
acc_clean['emp_title'].value_counts()

Unknown                                    43031
Teacher                                    10707
Manager                                     9461
Owner                                       5109
Registered Nurse                            4402
                                           ...  
CHIEF WARRANT OFFICER 4                        1
Associate Manager Corporate Audit              1
Highway Technician #3                          1
Director, The College Place-Virginia           1
University of Florida Smather's Library        1
Name: emp_title, Length: 217365, dtype: int64

In [66]:
acc_clean = acc_clean.drop('emp_title', axis=1)

### joint

In [67]:
# if no secondary app, convert to -1

acc_clean.loc[acc_clean['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1

acc_clean.loc[acc_clean['sec_app_fico_score'].isnull(), 'sec_app_fico_score'] = -1

### others

In [68]:
acc_clean.loc[acc_clean['annual_inc'].isnull(), 'annual_inc'] = -1

acc_clean.loc[acc_clean['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000

acc_clean.loc[acc_clean['duration_of_credit_months'].isnull(),
                  'duration_of_credit_months'] = -1

acc_clean.loc[acc_clean['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

acc_clean.loc[acc_clean['term'].str.contains('60', na=False), 'term'] = 1
acc_clean.loc[acc_clean['term'].str.contains('36', na=False), 'term'] = 0

acc_clean['term'] = acc_clean['term'].astype(int)

### total accounts

In [69]:
cols_w_acc = [col for col in acc_clean.columns if 'acc' in col]
acc_clean[cols_w_acc]

,open_acc,total_acc,acc_now_delinq,acc_open_past_24mths,mo_sin_old_il_acct,mort_acc,num_rev_accts,sec_app_mort_acc,sec_app_open_acc,sec_app_num_rev_accts
0,7.0,13.0,0.0,4.0,148.0,1.0,9.0,-1.0,-1.0,-1.0
1,12.0,27.0,0.0,6.0,142.0,4.0,16.0,-1.0,-1.0,-1.0
2,8.0,15.0,0.0,4.0,149.0,3.0,3.0,-1.0,-1.0,-1.0
3,14.0,23.0,0.0,7.0,164.0,1.0,19.0,-1.0,-1.0,-1.0
4,8.0,18.0,0.0,5.0,155.0,1.0,11.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
674045,12.0,22.0,0.0,3.0,14.0,0.0,20.0,-1.0,-1.0,-1.0
674046,18.0,28.0,0.0,12.0,110.0,0.0,24.0,-1.0,-1.0,-1.0
674047,15.0,38.0,1.0,6.0,124.0,7.0,17.0,-1.0,-1.0,-1.0
674048,11.0,24.0,0.0,3.0,82.0,6.0,13.0,-1.0,-1.0,-1.0


In [70]:
# checking if we can use open accounts for null values

acc_clean[(acc_clean['total_acc'].isnull()) & (acc_clean['open_acc'].isnull())][['total_acc',
                                                                                 'open_acc']]

,total_acc,open_acc


In [71]:
# using the open_acc as a filler for the null values for total_acc

acc_clean.loc[acc_clean['total_acc'].isnull(), 'total_acc'] = acc_clean['open_acc']

In [72]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

""
0
1
2
3
4
...
674045
674046
674047
674048


In [73]:
# getting the columns where there are 0s

cols_w_0s_and_null = []

for col in cols_w_null:
    acc_clean.loc[acc_clean[col_0].isnull(), col_0] = -1
    
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,0,13.99,123.03,3,14,10.0,MORTGAGE,55000.0,0,Dec-2015,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,148.0,677.0,-1.0,-1.0
1,20000.0,20000.0,20000.0,0,9.17,637.58,2,7,10.0,MORTGAGE,180000.0,0,Dec-2015,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,306.0,682.0,-1.0,-1.0
2,20000.0,20000.0,20000.0,0,8.49,631.26,2,6,10.0,MORTGAGE,85000.0,0,Dec-2015,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,202.0,707.0,-1.0,-1.0
3,10000.0,10000.0,10000.0,0,6.49,306.45,1,2,6.0,RENT,85000.0,0,Dec-2015,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,164.0,687.0,-1.0,-1.0
4,8000.0,8000.0,8000.0,0,11.48,263.74,2,10,10.0,MORTGAGE,42000.0,0,Dec-2015,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,253.0,702.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [74]:
acc_clean.to_csv("acc_1_train_basicimputation_predum.csv.gz",
                 index=False,
                 compression="gzip")

# Dummification

In [75]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_clean.select_dtypes(include=['object']).columns)

cols_w_obj

['home_ownership', 'issue_d', 'purpose', 'initial_list_status']

In [76]:
# dummification

acc_dum = pd.get_dummies(acc_clean, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue_d_Mar-2016,issue_d_Mar-2017,issue_d_Mar-2018,issue_d_May-2008,issue_d_May-2009,issue_d_May-2010,issue_d_May-2011,issue_d_May-2012,issue_d_May-2013,issue_d_May-2014,issue_d_May-2015,issue_d_May-2016,issue_d_May-2017,issue_d_May-2018,issue_d_Nov-2007,issue_d_Nov-2008,issue_d_Nov-2009,issue_d_Nov-2010,issue_d_Nov-2011,issue_d_Nov-2012,issue_d_Nov-2013,issue_d_Nov-2014,issue_d_Nov-2015,issue_d_Nov-2016,issue_d_Nov-2017,issue_d_Nov-2018,issue_d_Oct-2007,issue_d_Oct-2008,issue_d_Oct-2009,issue_d_Oct-2010,issue_d_Oct-2011,issue_d_Oct-2012,issue_d_Oct-2013,issue_d_Oct-2014,issue_d_Oct-2015,issue_d_Oct-2016,issue_d_Oct-2017,issue_d_Oct-2018,issue_d_Sep-2007,issue_d_Sep-2008,issue_d_Sep-2009,issue_d_Sep-2010,issue_d_Sep-2011,issue_d_Sep-2012,issue_d_Sep-2013,issue_d_Sep-2014,issue_d_Sep-2015,issue_d_Sep-2016,issue_d_Sep-2017,issue_d_Sep-2018,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,p

# Scaling the dataset

In [77]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt']])

acc_dum = acc_dum.drop('total_pymnt', axis=1)

In [78]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [79]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue_d_Mar-2016,issue_d_Mar-2017,issue_d_Mar-2018,issue_d_May-2008,issue_d_May-2009,issue_d_May-2010,issue_d_May-2011,issue_d_May-2012,issue_d_May-2013,issue_d_May-2014,issue_d_May-2015,issue_d_May-2016,issue_d_May-2017,issue_d_May-2018,issue_d_Nov-2007,issue_d_Nov-2008,issue_d_Nov-2009,issue_d_Nov-2010,issue_d_Nov-2011,issue_d_Nov-2012,issue_d_Nov-2013,issue_d_Nov-2014,issue_d_Nov-2015,issue_d_Nov-2016,issue_d_Nov-2017,issue_d_Nov-2018,issue_d_Oct-2007,issue_d_Oct-2008,issue_d_Oct-2009,issue_d_Oct-2010,issue_d_Oct-2011,issue_d_Oct-2012,issue_d_Oct-2013,issue_d_Oct-2014,issue_d_Oct-2015,issue_d_Oct-2016,issue_d_Oct-2017,issue_d_Oct-2018,issue_d_Sep-2007,issue_d_Sep-2008,issue_d_Sep-2009,issue_d_Sep-2010,issue_d_Sep-2011,issue_d_Sep-2012,issue_d_Sep-2013,issue_d_Sep-2014,issue_d_Sep-2015,issue_d_Sep-2016,issue_d_Sep-2017,issue_d_Sep-2018,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacat

In [80]:
col_names = list(acc_full.columns)
col_names.append('total_pymnt')

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue_d_Mar-2016,issue_d_Mar-2017,issue_d_Mar-2018,issue_d_May-2008,issue_d_May-2009,issue_d_May-2010,issue_d_May-2011,issue_d_May-2012,issue_d_May-2013,issue_d_May-2014,issue_d_May-2015,issue_d_May-2016,issue_d_May-2017,issue_d_May-2018,issue_d_Nov-2007,issue_d_Nov-2008,issue_d_Nov-2009,issue_d_Nov-2010,issue_d_Nov-2011,issue_d_Nov-2012,issue_d_Nov-2013,issue_d_Nov-2014,issue_d_Nov-2015,issue_d_Nov-2016,issue_d_Nov-2017,issue_d_Nov-2018,issue_d_Oct-2007,issue_d_Oct-2008,issue_d_Oct-2009,issue_d_Oct-2010,issue_d_Oct-2011,issue_d_Oct-2012,issue_d_Oct-2013,issue_d_Oct-2014,issue_d_Oct-2015,issue_d_Oct-2016,issue_d_Oct-2017,issue_d_Oct-2018,issue_d_Sep-2007,issue_d_Sep-2008,issue_d_Sep-2009,issue_d_Sep-2010,issue_d_Sep-2011,issue_d_Sep-2012,issue_d_Sep-2013,issue_d_Sep-2014,issue_d_Sep-2015,issue_d_Sep-2016,issue_d_Sep-2017,issue_d_Sep-2018,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacat

In [81]:
acc_full.to_csv("acc_1_train_basicimputation.csv.gz",
                index=False,
                compression="gzip")

target_1_train_basicimputation.to_csv("target_1_train_basicimputation.csv.gz",
                                       index=False,
                                       compression="gzip")

## Downsampling the dataset

In [82]:
from sklearn.utils import resample

label = target_1_train_basicimputation['loan_status_binary']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False,
                         n_samples=int(len(target_1_train_basicimputation.index)*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_full.iloc[combined]
target_subsample = target_1_train_basicimputation[['loan_status_binary']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

loan_status_binary    0.5
dtype: float64

In [83]:
acc_subsample

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue_d_Mar-2016,issue_d_Mar-2017,issue_d_Mar-2018,issue_d_May-2008,issue_d_May-2009,issue_d_May-2010,issue_d_May-2011,issue_d_May-2012,issue_d_May-2013,issue_d_May-2014,issue_d_May-2015,issue_d_May-2016,issue_d_May-2017,issue_d_May-2018,issue_d_Nov-2007,issue_d_Nov-2008,issue_d_Nov-2009,issue_d_Nov-2010,issue_d_Nov-2011,issue_d_Nov-2012,issue_d_Nov-2013,issue_d_Nov-2014,issue_d_Nov-2015,issue_d_Nov-2016,issue_d_Nov-2017,issue_d_Nov-2018,issue_d_Oct-2007,issue_d_Oct-2008,issue_d_Oct-2009,issue_d_Oct-2010,issue_d_Oct-2011,issue_d_Oct-2012,issue_d_Oct-2013,issue_d_Oct-2014,issue_d_Oct-2015,issue_d_Oct-2016,issue_d_Oct-2017,issue_d_Oct-2018,issue_d_Sep-2007,issue_d_Sep-2008,issue_d_Sep-2009,issue_d_Sep-2010,issue_d_Sep-2011,issue_d_Sep-2012,issue_d_Sep-2013,issue_d_Sep-2014,issue_d_Sep-2015,issue_d_Sep-2016,issue_d_Sep-2017,issue_d_Sep-2018,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacat

In [84]:
target_subsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
269355,1
269356,1
269357,1
269358,1


In [85]:
target_subsample.value_counts()

loan_status_binary
0                     134680
1                     134680
dtype: int64

In [86]:
acc_subsample.to_csv("acc_1_train_downsample.csv.gz",
                index=False,
                compression="gzip")

target_subsample.to_csv("target_1_train_downsample.csv.gz",
                        index=False,
                        compression="gzip")

In [87]:
grade_cols = [col for col in acc_subsample.columns if 'grade' in col]

grade_cols

['grade', 'sub_grade']

In [88]:
acc_subsample_nogrades = acc_subsample.drop(grade_cols, axis=1)

acc_subsample_nogrades

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,verification_status,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue_d_Mar-2016,issue_d_Mar-2017,issue_d_Mar-2018,issue_d_May-2008,issue_d_May-2009,issue_d_May-2010,issue_d_May-2011,issue_d_May-2012,issue_d_May-2013,issue_d_May-2014,issue_d_May-2015,issue_d_May-2016,issue_d_May-2017,issue_d_May-2018,issue_d_Nov-2007,issue_d_Nov-2008,issue_d_Nov-2009,issue_d_Nov-2010,issue_d_Nov-2011,issue_d_Nov-2012,issue_d_Nov-2013,issue_d_Nov-2014,issue_d_Nov-2015,issue_d_Nov-2016,issue_d_Nov-2017,issue_d_Nov-2018,issue_d_Oct-2007,issue_d_Oct-2008,issue_d_Oct-2009,issue_d_Oct-2010,issue_d_Oct-2011,issue_d_Oct-2012,issue_d_Oct-2013,issue_d_Oct-2014,issue_d_Oct-2015,issue_d_Oct-2016,issue_d_Oct-2017,issue_d_Oct-2018,issue_d_Sep-2007,issue_d_Sep-2008,issue_d_Sep-2009,issue_d_Sep-2010,issue_d_Sep-2011,issue_d_Sep-2012,issue_d_Sep-2013,issue_d_Sep-2014,issue_d_Sep-2015,issue_d_Sep-2016,issue_d_Sep-2017,issue_d_Sep-2018,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedd

In [89]:
acc_subsample_nogrades.to_csv("acc_1_train_downsample_nogrades.csv.gz",
                              index=False,
                              compression="gzip")